In [1]:
signature STACK =
sig
  type 'a Stack
  val empty : 'a Stack
  val isEmpty : 'a Stack -> bool
  val cons : 'a * 'a Stack -> 'a Stack
  val head : 'a Stack -> 'a
  val tail : 'a Stack -> 'a Stack
end;

signature STACK =
  sig
    type 'a Stack
    val empty : 'a Stack
    val isEmpty : 'a Stack -> bool
    val cons : 'a * 'a Stack -> 'a Stack
    val head : 'a Stack -> 'a
    val tail : 'a Stack -> 'a Stack
  end


In [2]:
structure List : STACK =
struct
  type 'a Stack = 'a list
  val empty = []
  fun isEmpty s = null s
  fun cons (x, s) = x :: s
  fun head s = hd s
  fun tail s = tl s
end;

structure List : STACK


In [3]:
structure CustomStack : STACK =
struct
  datatype 'a Stack = NIL | CONS of 'a * 'a Stack
  val empty = NIL
  fun isEmpty NIL = true | isEmpty _ = false
  fun cons (x, s) = CONS (x, s)
  fun head NIL = raise Empty
    | head (CONS (x, s)) = x
  fun tail NIL = raise Empty
    | tail (CONS (x, s)) = s
end;

structure CustomStack : STACK


In [4]:
local
  open CustomStack
in
  fun append (xs : 'a Stack)  (ys : 'a Stack) = if isEmpty xs then ys else cons (head xs, append (tail xs) ys)
end;

val append = fn
  : 'a CustomStack.Stack -> 'a CustomStack.Stack -> 'a CustomStack.Stack


In [5]:
fun append [] ys = ys
  | append (x :: xs) ys = x :: (append xs ys);

val append = fn : 'a list -> 'a list -> 'a list


In [6]:
fun update ([], i, y) = raise Subscript
  | update (x :: xs, 0, y) = y :: xs
  | update (x :: xs, i, y) = x :: update (xs, i-1, y);

val update = fn : 'a list * int * 'a -> 'a list


In [7]:
(* ex.2.1 *)
fun suffixes [] = [[]]
  | suffixes (l as (x :: xs)) = l :: suffixes xs;

val suffixes = fn : 'a list -> 'a list list


In [8]:
suffixes [1,2,3,4];

val it = [[1,2,3,4],[2,3,4],[3,4],[4],[]] : int list list


In [9]:
signature SET =
sig
  type Elem
  type Set
  val empty : Set
  val insert : Elem * Set -> Set
  val member : Elem * Set -> bool
end;

signature SET =
  sig
    type Elem
    type Set
    val empty : Set
    val insert : Elem * Set -> Set
    val member : Elem * Set -> bool
  end


In [10]:
(* Fig. 2.9 *)
signature ORDERED =
sig
  type T
  val eq : T * T -> bool
  val lt : T * T -> bool
  val leq : T * T -> bool
end;

functor UnbalancedSet (Element : ORDERED) : SET =
struct
  type Elem = Element.T
  datatype Tree = E | T of Tree * Elem * Tree
  type Set = Tree
  val empty = E
  fun member (x, E) = false
    | member (x, T (a, y, b)) = 
        if Element.lt (x, y) then member (x, a)
        else if Element.lt (y, x) then member (x, b)
        else true
  fun insert (x, E) = T (E, x, E)
    | insert (x, s as T (a, y, b)) =
        if Element.lt (x, y) then T (insert (x, a), y, b)
        else if Element.lt (y, x) then T (a, y, insert (x, b))
        else s
end;

signature ORDERED =
  sig
    type T
    val eq : T * T -> bool
    val lt : T * T -> bool
    val leq : T * T -> bool
  end
functor UnbalancedSet(Element: sig
                                 type T
                                 val eq : T * T -> bool
                                 val lt : T * T -> bool
                                 val leq : T * T -> bool
                               end) :
                     sig
                       type Elem
                       type Set
                       val empty : Set
                       val insert : Elem * Set -> Set
                       val member : Elem * Set -> bool
                     end


In [11]:
structure IntElement : ORDERED =
struct 
  type T = int
  fun eq (x, y) = x = y
  fun lt (x, y) = x < y
  fun leq (x, y) = x <= y
end;

stdIn:11.77 Warning: calling polyEqual
structure IntElement : ORDERED


In [12]:
structure IntTree = UnbalancedSet (IntElement);
val t1 = IntTree.insert (6, IntTree.insert (3, IntTree.insert (1, IntTree.insert (2, IntTree.insert (5, IntTree.empty)))));
IntTree.member (1, t1);
IntTree.member (4, t1);
IntTree.member (4, IntTree.insert (4, t1));

structure IntTree : SET
val t1 = T (T (T #,2,T #),5,T (E,6,E)) : IntTree.Set
val it = true : bool
val it = false : bool
val it = true : bool


In [13]:
(* ex.2.2 *)
functor UnbalancedSet (Element : ORDERED) : SET =
struct
  type Elem = Element.T
  datatype Tree = E | T of Tree * Elem * Tree
  type Set = Tree
  val empty = E
  fun member (x, t) = let
    fun member_rec (x, E, NONE) = false
      | member_rec (x, E, SOME z) = Element.eq (x, z)
      | member_rec (x, T (a, y, b), z) = 
          if Element.lt (x, y) then member_rec (x, a, z)
          else member_rec (x, b, SOME y)
  in member_rec (x, t, NONE) end
  fun insert (x, E) = T (E, x, E)
    | insert (x, s as T (a, y, b)) =
        if Element.lt (x, y) then T (insert (x, a), y, b)
        else if Element.lt (y, x) then T (a, y, insert (x, b))
        else s
end;

functor UnbalancedSet(Element: sig
                                 type T
                                 val eq : T * T -> bool
                                 val lt : T * T -> bool
                                 val leq : T * T -> bool
                               end) :
                     sig
                       type Elem
                       type Set
                       val empty : Set
                       val insert : Elem * Set -> Set
                       val member : Elem * Set -> bool
                     end


In [14]:
structure IntTree = UnbalancedSet (IntElement);
val t1 = IntTree.insert (6, IntTree.insert (3, IntTree.insert (1, IntTree.insert (2, IntTree.insert (5, IntTree.empty)))));
IntTree.member (1, t1);
IntTree.member (4, t1);

structure IntTree : SET
val t1 = T (T (T #,2,T #),5,T (E,6,E)) : IntTree.Set
val it = true : bool
val it = false : bool


In [15]:
(* ex.2.3 *)
functor UnbalancedSet (Element : ORDERED) : SET =
struct
  type Elem = Element.T
  datatype Tree = E | T of Tree * Elem * Tree
  type Set = Tree
  val empty = E
  fun member (x, t) = let
    fun member_rec (x, E, NONE) = false
      | member_rec (x, E, SOME z) = Element.eq (x, z)
      | member_rec (x, T (a, y, b), z) = 
          if Element.lt (x, y) then member_rec (x, a, z)
          else member_rec (x, b, SOME y)
  in member_rec (x, t, NONE) end
  exception ExistElem
  fun insert (x, t) = let
    fun insert_rec (x, E) = T (E, x, E)
      | insert_rec (x, s as T (a, y, b)) =
          if Element.lt (x, y) then T (insert_rec (x, a), y, b)
          else if Element.lt (x, y) then T (a, y, insert_rec (x, b))
          else raise ExistElem
  in insert_rec (x, t) handle ExistElem => t end
end;

functor UnbalancedSet(Element: sig
                                 type T
                                 val eq : T * T -> bool
                                 val lt : T * T -> bool
                                 val leq : T * T -> bool
                               end) :
                     sig
                       type Elem
                       type Set
                       val empty : Set
                       val insert : Elem * Set -> Set
                       val member : Elem * Set -> bool
                     end


In [16]:
(* ex.2.4 *)
functor UnbalancedSet (Element : ORDERED) : SET =
struct
  type Elem = Element.T
  datatype Tree = E | T of Tree * Elem * Tree
  type Set = Tree
  val empty = E
  fun member (x, t) = let
    fun member_rec (x, E, NONE) = false
      | member_rec (x, E, SOME z) = Element.eq (x, z)
      | member_rec (x, T (a, y, b), z) = 
          if Element.lt (x, y) then member_rec (x, a, z)
          else member_rec (x, b, SOME y)
  in member_rec (x, t, NONE) end
  exception ExistElem
  fun insert (x, t) = let
    fun insert_rec (x, E, NONE) = T (E, x, E)
      | insert_rec (x, E, SOME z) = 
          if Element.eq (x, z) then raise ExistElem
          else T (E, x, E)
      | insert_rec (x, s as T (a, y, b), z) =
          if Element.lt (x, y) then T (insert_rec (x, a, z), y, b)
          else T (a, y, insert_rec (x, b, SOME y))
  in insert_rec (x, t, NONE) handle ExistElem => t end
end;

functor UnbalancedSet(Element: sig
                                 type T
                                 val eq : T * T -> bool
                                 val lt : T * T -> bool
                                 val leq : T * T -> bool
                               end) :
                     sig
                       type Elem
                       type Set
                       val empty : Set
                       val insert : Elem * Set -> Set
                       val member : Elem * Set -> bool
                     end


In [17]:
structure IntTree = UnbalancedSet (IntElement);
val t1 = IntTree.insert (6, IntTree.insert (3, IntTree.insert (1, IntTree.insert (2, IntTree.insert (5, IntTree.empty)))));
IntTree.member (1, t1);
IntTree.member (4, t1);
IntTree.member (4, IntTree.insert (4, t1));
IntTree.member (1, IntTree.insert (1, t1));

structure IntTree : SET
val t1 = T (T (T #,2,T #),5,T (E,6,E)) : IntTree.Set
val it = true : bool
val it = false : bool
val it = true : bool
val it = true : bool


In [18]:
(* ex.2.5 *)
signature SET =
sig
  type Elem
  type Set
  val empty : Set
  val insert : Elem * Set -> Set
  val member : Elem * Set -> bool
  val complete : Elem * int -> Set
  val balanced : Elem * int -> Set
end;

functor UnbalancedSet (Element : ORDERED) : SET =
struct
  type Elem = Element.T
  datatype Tree = E | T of Tree * Elem * Tree
  type Set = Tree
  val empty = E
  fun member (x, t) = let
    fun member_rec (x, E, NONE) = false
      | member_rec (x, E, SOME z) = Element.eq (x, z)
      | member_rec (x, T (a, y, b), z) = 
          if Element.lt (x, y) then member_rec (x, a, z)
          else member_rec (x, b, SOME y)
  in member_rec (x, t, NONE) end
  exception ExistElem
  fun insert (x, t) = let
    fun insert_rec (x, E, NONE) = T (E, x, E)
      | insert_rec (x, E, SOME z) = 
          if Element.eq (x, z) then raise ExistElem
          else T (E, x, E)
      | insert_rec (x, s as T (a, y, b), z) =
          if Element.lt (x, y) then T (insert_rec (x, a, z), y, b)
          else T (a, y, insert_rec (x, b, SOME y))
  in insert_rec (x, t, NONE) handle ExistElem => t end

  (* ex.2.5 (a) *)
  fun complete (x, 0) = E
    | complete (x, d) = let
        val t = complete (x, d - 1)
      in T (t, x, t) end

  (* ex.2.5 (b) *)
  fun balanced (x, 0) = E
    | balanced (x, 1) = T (E, x, E)
    | balanced (x, n) = let
        val m = (n - 1) div 2
      in T (balanced (x, m + ((n - 1) mod 2)), x, balanced (x, m)) end
end;

signature SET =
  sig
    type Elem
    type Set
    val empty : Set
    val insert : Elem * Set -> Set
    val member : Elem * Set -> bool
    val complete : Elem * int -> Set
    val balanced : Elem * int -> Set
  end
functor UnbalancedSet(Element: sig
                                 type T
                                 val eq : T * T -> bool
                                 val lt : T * T -> bool
                                 val leq : T * T -> bool
                               end) :
                     sig
                       type Elem
                       type Set
                       val empty : Set
                       val insert : Elem * Set -> Set
                       val member : Elem * Set -> bool
                       val complete : Elem * int -> Set
                       val balanced : Elem * int -> Set
                     end


In [19]:
structure IntTree = UnbalancedSet (IntElement);
IntTree.complete (2, 3);
IntTree.balanced (2, 4);

structure IntTree : SET
val it = T (T (T #,2,T #),2,T (T #,2,T #)) : IntTree.Set
val it = T (T (T #,2,E),2,T (E,2,E)) : IntTree.Set


In [20]:
(* ex.2.6 *)
signature FINITE_MAP =
sig
  type Key
  type 'a Map
  val empty : 'a Map
  val bind : Key * 'a * 'a Map -> 'a Map
  val lookup : Key * 'a Map -> 'a
end;

functor UnbalancedMap (KeyOrd : ORDERED) : FINITE_MAP =
struct
  type Key = KeyOrd.T
  datatype 'a MapTree = MapE | MapT of 'a MapTree * Key * 'a * 'a MapTree
  type 'a Map = 'a MapTree
  val empty = MapE
  exception NotFound
  fun bind (k, v, MapE) = MapT (MapE, k, v, MapE)
    | bind (k, v, MapT (a, kk, vv, b)) =
        if KeyOrd.lt (k, kk) then MapT (bind (k, v, a), kk, vv, b)
        else if KeyOrd.lt (kk, k) then MapT (a, kk, vv, bind (k, v, b))
        else MapT (a, k, v, b)
  fun lookup (k, MapE) = raise NotFound
    | lookup (k, MapT (a, kk, vv, b)) =
        if KeyOrd.lt (k, kk) then lookup (k, a) 
        else if KeyOrd.lt (kk, k) then lookup (k, b)
        else vv
end;

signature FINITE_MAP =
  sig
    type Key
    type 'a Map
    val empty : 'a Map
    val bind : Key * 'a * 'a Map -> 'a Map
    val lookup : Key * 'a Map -> 'a
  end
functor UnbalancedMap(KeyOrd: sig
                                type T
                                val eq : T * T -> bool
                                val lt : T * T -> bool
                                val leq : T * T -> bool
                              end) :
                     sig
                       type Key
                       type 'a Map
                       val empty : 'a Map
                       val bind : Key * 'a * 'a Map -> 'a Map
                       val lookup : Key * 'a Map -> 'a
                     end


In [21]:
structure IntMap = UnbalancedMap (IntElement);
val t1 = IntMap.bind (1, "a", IntMap.empty);
val t2 = IntMap.bind (5, "d", IntMap.bind (3, "c", IntMap.bind (2, "b", t1)));
IntMap.lookup (2, t2);
IntMap.lookup (4, t2);

structure IntMap : FINITE_MAP
val t1 = MapT (MapE,1,"a",MapE) : string IntMap.Map
val t2 = MapT (MapE,1,"a",MapT (MapE,2,"b",MapT #)) : string IntMap.Map
val it = "b" : string

uncaught exception NotFound
  raised at: stdIn:20.685-20.693
